In [1]:
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.wrappers import Request, Response
import sqlite3

In [2]:
dataBase= "DataBase/db_base.db"

class DB:
    def ejecutar_consulta(self, consulta, parametros = ()):
        with sqlite3.connect(dataBase) as conn:
            self.cursor = conn.cursor()
            result = self.cursor.execute(consulta, parametros)
            conn.commit()
            return result

In [3]:
app = Flask(__name__)

In [4]:
db=DB()

@app.route('/')
@app.route('/home')
def home():
    id_cliente= request.args.get('id_cliente')
    nameUser = request.args.get('usuario')
    productos = db.ejecutar_consulta("SELECT * FROM producto")
    listaProductos = list()
    for producto in productos:
        listaProductos.append(producto)
    return render_template("index.html", listaProductos=listaProductos, nameUser=nameUser, id_cliente=id_cliente)

@app.route('/register')
def registrar():
    return render_template("persona.html")

@app.route('/update')
def update():
    return render_template("update.html")

@app.route('/create_person', methods=['POST'])
def create_Person():
    
    if request.method == 'POST':
        nombre = request.form.get('nombre')
        direccion = request.form.get('direccion')
        telefono = request.form.get('telefono')
        email = request.form.get('email')
        
        sql = "INSERT INTO cliente (nombre,direccion,telefono,email) VALUES(?,?,?,?)"
        parametros = (nombre,direccion,telefono,email)
        db.ejecutar_consulta(sql,parametros)
    
        return redirect(url_for('home'))
    
@app.route('/search')
@app.route('/search', methods=['POST'])
def buscar():
    if request.method == 'POST':
        id_cliente = request.form.get('buscar')
        sql="SELECT * FROM cliente WHERE id=?"
        parametros=[id_cliente,]
        cliente = db.ejecutar_consulta(sql,parametros)
        for id_c in cliente:
            if id_c[0] != 0:
                estado = "disabled"
                return render_template("persona.html", dato=str(id_c[0]))
            else:
                print("no")
                
@app.route('/searchA')
@app.route('/searchA', methods=['POST'])
def buscarA():
    if request.method == 'POST':
        id_cliente = request.form.get('buscar')
        sql="SELECT id FROM cliente WHERE id=?"
        parametros=[id_cliente,]
        cliente = db.ejecutar_consulta(sql,parametros)
        for id_c in cliente:
            if id_c[0] != 0:
                return render_template("update.html", dato=str(id_c[0]))
            else:
                print("no")

@app.route('/delete')
@app.route('/delete/<int:id>')
def delete(id):
    if(id !=0):
        sql="DELETE FROM cliente WHERE id=?"
        parametros = (id,)
        pragma="PRAGMA foreign_keys=ON"
        db.ejecutar_consulta(pragma,)
        db.ejecutar_consulta(sql,parametros)
        return 'Eliminado...'
    
@app.route('/upDate')
@app.route('/upDate/<int:id>',methods=['POST'])
def upDate(id):
    if request.method == 'POST':
        nombre = request.form.get('nombre')
        direccion = request.form.get('direccion')
        telefono = request.form.get('telefono')
        email = request.form.get('email')

        sql="UPDATE cliente SET nombre=?,direccion=?,telefono=?,email=? WHERE id=?"
        parametros = (nombre,direccion,telefono,email,id)
        db.ejecutar_consulta(sql,parametros)
        return redirect(url_for('home'))


@app.route('/seleccionar')
@app.route('/seleccionar/<string:name>/<int:id_cliente>/<int:id_producto>')
def seleccionar(name=None, id_producto=None, id_cliente=None):
    
    if (name!= None and id_producto != None and id_cliente!=None):
        sql_targeta = '''
        SELECT *
        FROM targeta
        INNER JOIN cuenta
        ON targeta.id=cuenta.id_targeta
        WHERE cuenta.id_cliente=?
        '''
        sql = "SELECT valor,id FROM producto WHERE id=?"
        parametros = [id_producto,]
        parametrosTar=[id_cliente,]
        productos = db.ejecutar_consulta(sql,parametros)
        targetas = db.ejecutar_consulta(sql_targeta,parametrosTar)
        
        listaTargetas=list()
        
        for producto in productos:
            pro_Precio = producto
            pro_Id= producto[1]
            
        for targeta in targetas:
            print(targeta[0],targeta[1])
            listaTargetas.append(targeta)
             
        return render_template('targeta.html', usuario=name,pro_Id=pro_Id, id_cliente=id_cliente, listaTargetas=listaTargetas)
        

@app.route('/pago/<int:id_producto>/<int:id_usuario>', methods=['POST'])
def pago(id_producto=None, id_usuario=None):
    if request.method=="POST":
        if id_producto != None and id_usuario != None:
            sql = '''
            SELECT id_cliente, nom_targeta, saldo, descripcion, valor
            FROM targeta
            INNER JOIN cuenta, producto
            ON targeta.id=cuenta.id_targeta
            WHERE cuenta.id_targeta=?
            AND producto.id=?
            AND cuenta.id_cliente=?
            '''
            id_targeta = request.form.get("opcion")
            parametros = [id_targeta,id_producto, id_usuario]
            listaClientes = db.ejecutar_consulta(sql,parametros)
            print(parametros)
            listaCliente=list()
            for clientes in listaClientes:
                listaCliente.append(clientes)
            return render_template('factura.html', listaCliente=listaCliente, id_producto=id_producto)
    
@app.route('/sing_in')
def sing_in():
    return render_template('sing_in.html')
    
@app.route('/consulta', methods=['POST'])
def consulta():
    if request.method=='POST':
        email = request.form.get('email')
        sql = 'SELECT * FROM cliente WHERE email=?'
        parametros=[email,]
        clientes = db.ejecutar_consulta(sql,parametros)
        for cliente in clientes:
            id_cliente = cliente[0]
            nombreCliente = cliente[1]
            print(nombreCliente)
            return redirect(url_for('home', usuario=nombreCliente, id_cliente=id_cliente))

@app.route('/cancel/<string:user>')
def calcel(user=None):
    if user != None:
        sql = '''
        SELECT * FROM cliente WHERE nombre=?
        '''
        parametros = [user,]
        usuario = db.ejecutar_consulta(sql,parametros)
        for u in usuario:
            return redirect(url_for('home',usuario=u[1], id_cliente=u[0]))

@app.route('/buy/<int:id_producto>/<int:cantidad>')
def buy(id_producto=None, cantidad=None):
    
    if id_producto != None and cantidad != None:
        sql = 'UPDATE producto SET stock = stock - 1 WHERE producto.id=?'
        parametros=[id_producto,]
        db.ejecutar_consulta(sql,parametros)
        return 'Compra exitosa'
    
@app.route('/error')
def error():
    return render_template('error.html')

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001,app)

 * Running on http://localhost:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2021 17:56:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:34] "GET /error HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:37] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:39] "GET /error HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:41] "GET /sing_in HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:43] "POST /consulta HTTP/1.1" 302 -


Wilson


127.0.0.1 - - [25/May/2021 17:56:43] "GET /home?usuario=Wilson&id_cliente=4 HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:56:47] "GET /seleccionar/Wilson/4/8 HTTP/1.1" 200 -


4 American Express
3 Mastercard


127.0.0.1 - - [25/May/2021 17:56:50] "POST /pago/8/4 HTTP/1.1" 200 -


['4', 8, 4]


127.0.0.1 - - [25/May/2021 17:57:03] "GET /buy/8/1 HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:57:24] "GET / HTTP/1.1" 200 -
